In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, set_seed
from datasets import *
import numpy as np
import pandas as pd
import torch
import ast
import evaluate
import os

In [2]:
local_models_path = '../data/models/T5/SE'

results = pd.DataFrame(columns=['Model', 'Data Type', 'Bert.Precision', 'Bert.Recall', 'Bert.F1', 'BLEU'])

In [3]:
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")

In [4]:
def get_answer(question, context, tokenizer, model):
    input_text = "question: %s  context: %s" % (question, context)
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [5]:
models = os.listdir(local_models_path)

In [6]:
handwritten = [model for model in models if 'handwritten' in model]
rest = [model for model in models if 'handwritten' not in model]

In [7]:
model = None
tokenizer = None

for model in rest:
    model_data = model.split("-")

    print(f"Evaluating model {model}...")
    model_name = f"{local_models_path}/{model}"
    seed = model_data[-1]
    set_seed(int(seed))

    data_2020_2022 = load_dataset('csv', data_files="../data/clean/sustainability-report-2042-squad-format.csv",
                                delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=int(seed))

    # Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
    data_2020_2022["test"] = data_2020_2022["test"].map(
        lambda example: ast.literal_eval(example["answers"]))
    data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                    "text": example["text"], "answer_start": example["answer_start"]}})
    # replace all "\n" with " " in the context, answers and questions
    data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                    "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
    data_2020_2022["test"] = data_2020_2022["test"].remove_columns(["text", "answer_start"])
    # get ground truth answers
    test_data_2020_2022 = data_2020_2022["test"]
    gt_answers_2020_2022 = [temp["answers"]["text"][0] for temp in test_data_2020_2022]

    tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, local_files_only=True)

    answers_2020_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]

    bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
    bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)

    results.loc[len(results)] = ['t5-small', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
    
    del tokenizer
    del model
    torch.cuda.empty_cache()

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined-42...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-a6bdc04297c1a3c2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-4c58d6607e064ca8.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-6ec98b5ad96d608a.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b60d97fbf617fa23.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-da9287d5acd53d81.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined-43...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-a6bdc04297c1a3c2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-979a8191f09e1035.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-82741fa36c858cca.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-d3b0fec5fe205d53.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c40597fb67e48763.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined-44...


...

Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-a6bdc04297c1a3c2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-d34a0b1b11ab4a95.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b665426520fe3b88.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-dbd4ef42eb1f3d16.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c01f07850c451399.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined-50...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-a6bdc04297c1a3c2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-2d1d36e1fafcd624.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-e5a3431c3645914a.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ab781c79497628ff.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-39c0a9da5a394f05.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined-51...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-a6bdc04297c1a3c2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-3bdefb8b746cf12d.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-d0ac775eb9473ec2.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ba63ee62d3e2d4d2.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-a6bdc04297c1a3c2\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-2ebfcc63cf8ec15c.arrow
Loading cached processed da

In [8]:
tokenizer = None
model = None

for model in handwritten:
    model_data = model.split("-")

    print(f"Evaluating model {model}...")
    model_name = f"{local_models_path}/{model}"
    seed = model_data[-1]
    set_seed(int(seed))

    data_2022_handwritten = load_dataset('csv', data_files=f"../data/clean/QA_SR_2022_Expert-squad-format.csv",
                                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=int(seed))

    # Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
    data_2022_handwritten["test"] = data_2022_handwritten["test"].map(
        lambda example: ast.literal_eval(example["answers"]))
    data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                    "text": example["text"], "answer_start": example["answer_start"]}})
    # replace all "\n" with " " in the context, answers and questions
    data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                    "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
    data_2022_handwritten["test"] = data_2022_handwritten["test"].remove_columns(["text", "answer_start"])

    test_data_2022_handwritten = data_2022_handwritten["test"]
    gt_answers_2022_handwritten = [temp["answers"]["text"][0] for temp in test_data_2022_handwritten]

    tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, local_files_only=True)

    answers_2022_handwritten = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

    bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")
    bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)

    
    results.loc[len(results)] = ['t5-small', 'handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]
     
    del tokenizer
    del model
    torch.cuda.empty_cache()

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten-42...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-853b320bab41342e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-690d708f98f9a3b4.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b3c2cbaa1563558c.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-227bc1b6d7ce66b1.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ea73fd3146b3fd5a.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten-43...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-853b320bab41342e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c2e5e8fd88c0fa07.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-1c0de4013f019b4a.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-5576838a6f55ab89.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-d0041066c4ca1a8f.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten-44...


...

Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-853b320bab41342e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-f127e20bfb0e4faf.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-be2b029fbc7bc3bf.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-9f4c17020c08c45c.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-871d9beeb4c25ea4.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten-50...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-853b320bab41342e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-31f7281be09e829a.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-8dcdbdb5507f9b95.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-966f5da43e073ed3.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ed8d1e5688f5f911.arrow
Loading cached processed da

Evaluating model t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten-51...


Found cached dataset csv (C:/Users/Luka/.cache/huggingface/datasets/csv/default-853b320bab41342e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-18648557ffcf97ce.arrow and C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c4b56581efe1a940.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b679643ee972cfa3.arrow
Loading cached processed dataset at C:\Users\Luka\.cache\huggingface\datasets\csv\default-853b320bab41342e\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-e76297182368c96b.arrow
Loading cached processed da

In [9]:
df_2020_2022 = results[results['Data Type'] == '2020-2022']
df_handwritten = results[results['Data Type'] == 'handwritten']
display(df_2020_2022)
display(df_handwritten)

,Model,Data Type,Bert.Precision,Bert.Recall,Bert.F1,BLEU
0,t5-small,2020-2022,0.968892,0.981904,0.975069,0.543063
1,t5-small,2020-2022,0.972492,0.979039,0.975480,0.531009
2,t5-small,2020-2022,0.963853,0.972613,0.967849,0.578645
3,t5-small,2020-2022,0.973157,0.980202,0.976383,0.614047
4,t5-small,2020-2022,0.966682,0.973360,0.969763,0.562466
5,t5-small,2020-2022,0.973078,0.982073,0.977263,0.626648
6,t5-small,2020-2022,0.974937,0.978690,0.976623,0.617954
7,t5-small,2020-2022,0.970898,0.978491,0.974389,0.570040
8,t5-small,2020-2022,0.966904,0.977225,0.971779,0.555747
9,t5-small,2020-2022,0.967508,0.974424,0.970657,0.573230


,Model,Data Type,Bert.Precision,Bert.Recall,Bert.F1,BLEU
10,t5-small,handwritten,0.911456,0.878175,0.894139,0.221080
11,t5-small,handwritten,0.930275,0.894927,0.911898,0.293840
12,t5-small,handwritten,0.911231,0.876724,0.893213,0.114651
13,t5-small,handwritten,0.920100,0.893000,0.905997,0.110058
14,t5-small,handwritten,0.928027,0.900291,0.913582,0.243488
15,t5-small,handwritten,0.924586,0.893090,0.908213,0.185495
16,t5-small,handwritten,0.936590,0.899866,0.917407,0.308283
17,t5-small,handwritten,0.901736,0.862724,0.881626,0.193429
18,t5-small,handwritten,0.906286,0.875989,0.890618,0.128802
19,t5-small,handwritten,0.908682,0.873197,0.890186,0.166017


In [10]:
# get average results and standard error
df_2020_2022_mean = df_2020_2022.mean(axis=0)
df_2020_2022_sem = df_2020_2022.sem(axis=0)
df_2020_2022_std = df_2020_2022.std(axis=0)

df_handwritten_mean = df_handwritten.mean(axis=0)
df_handwritten_sem = df_handwritten.sem(axis=0)
df_handwritten_std = df_handwritten.std(axis=0)

C:\Users\Luka\AppData\Local\Temp\ipykernel_13708\3485297047.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_2020_2022_mean = df_2020_2022.mean(axis=0)
C:\Users\Luka\AppData\Local\Temp\ipykernel_13708\3485297047.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_2020_2022_sem = df_2020_2022.sem(axis=0)
C:\Users\Luka\AppData\Local\Temp\ipykernel_13708\3485297047.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_2020_2022_std = df_2020_2022.std(axis=0)
C:\Users\Luka\AppDat

In [11]:
# display results
print("2020-2022")
print("Mean")
display(df_2020_2022_mean)
print("Standard Error")
display(df_2020_2022_sem)
print("Standard Deviation")
display(df_2020_2022_std)

print()

print("HANDWRITTEN")
print("Mean")
display(df_handwritten_mean)
print("Standard Error")
display(df_handwritten_sem)
print("Standard Deviation")
display(df_handwritten_std)

2020-2022
Mean


Bert.Precision    0.969840
Bert.Recall       0.977802
Bert.F1           0.973526
BLEU              0.577285
dtype: float64

Standard Error


Bert.Precision    0.001137
Bert.Recall       0.001065
Bert.F1           0.001035
BLEU              0.010279
dtype: float64

Standard Deviation


...

Bert.Precision    0.917897
Bert.Recall       0.884798
Bert.F1           0.900688
BLEU              0.196514
dtype: float64

Standard Error


Bert.Precision    0.003684
Bert.Recall       0.004102
Bert.F1           0.003844
BLEU              0.022260
dtype: float64

Standard Deviation


Bert.Precision    0.011651
Bert.Recall       0.012971
Bert.F1           0.012157
BLEU              0.070393
dtype: float64